In [61]:
%pip install PyPDF2
%pip install regex
%pip install sentence-transformers
%pip install faiss-cpu
%pip install openai
%pip install google.generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Extract Text from PDF

In [66]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Example usage:
pdf_text = extract_text_from_pdf("input.pdf")
# print(pdf_content)

## Split the Text into Chunks

In [67]:
import re

def chunk_text(text, max_chunk_size=500):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks, current_chunk = [], []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence)
        if current_length + sentence_length <= max_chunk_size:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

chunks = chunk_text(pdf_text)

In [68]:
chunks

[' STUDENT RESOURCE BOOK \n1Contents 1.Message from the Vice Chancellor 2.Message from the Director 3.Deﬁni9ons 4.Admission 5.Academics 6.Examina9on & Evalua9on 7.Placement Assistance 8.Student Support Services Guidelines 9.People you should know 2 Message from the Vice Chancellor Dr. Ramesh Bhat I take this opportunity of congratula9ng all of you on your decision to join NMIMS Global Access School for Con9nuing Educa9on (NGA – SCE) of NMIMS (Deemed – to - be – University.',
 'I am happy that you have joined the growing community of learners at NMIMS. Shri Vile Parle Kelavani Mandal, a Public Trust and a Society in Mumbai since 1934 whose goal is to advance the cause of quality educa9on in India, set up Narsee Monjee Ins9tute of Management Studies (NMIMS) in 1981 at the ini9a9on of University of Bombay (now University of Mumbai). By 1994-95, NMIMS had emerged as one of the top 10 B-Schools of India.',
 'Since then it has maintained its leadership posi9on in Management Educa9on in India

## Embed the Text Chunks

In [69]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
chunk_embeddings = model.encode(chunks)

## Store Embeddings for Retrieval

In [70]:
import faiss
import numpy as np

dimension = chunk_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)

# Add the chunk embeddings to the index
index.add(np.array(chunk_embeddings))

## Retrieve Relevant Chunks Based on User Query

In [83]:
def find_relevant_chunk(query, index, chunks, model, top_k=15):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunks[i] for i in indices[0]]

user_query = "What is the purpose of the document?"
relevant_chunks = find_relevant_chunk(user_query, index, chunks, model)

print("Relevant Chunk:", relevant_chunks[0])

Relevant Chunk: Students are expected to download the applicable subject latest assignment ques9on paper & submit the assignment on or before the last date announced by NGASCE for that respec9ve exam cycle. b.Internal Assignment Structure The purpose of the internal assessment is to evaluate the student’s understanding of concepts and applica9on thereof. Assignments are set to evaluate the student’s thought process, conceptual understanding and applica9on thereof.


## Expand the current Query

In [ ]:
import requests

def generate_answer_with_gemini(system_prompt, query, api_key):
    # print('Context: ', context)
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json",
        # "Authorization": f"Bearer {api_key}"
    }
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"SYSTEM_PROMPT: {system_prompt}\n\n USER'S_QUESTION: {query}\n\n EXPANDED_QUESTION:"
                    }
                ]
            }
        ],
        "generation_config": {
            "temperature": 0.0
        }
    }
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json().get('candidates')[0].get('content').get('parts')[0].get('text',"Sorry, I didn’t understand your question. Do you want to connect with a live agent?")
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
api_key = "api_key"
# failure_statement = "Sorry, I didn't understand your question. Do you want to connect with a live agent?"
System_Prompt = f"You are an AI Agent to help retrieve information from Student Resource Book of the NMIMS Global Access School For Continuing Education to answer user questions on that database. Expand the user's query and involve all the relevant questions you would want retrieve from the database to answer the user's question. I will provide you with the user's question, give me the expanded question."
# relevant_chunk = find_relevant_chunk(user_query, index, chunks, model)
user_query = input()

answer = generate_answer_with_gemini(System_Prompt, user_query, api_key)
print("Answer:", answer)

Answer: What are the admission criteria for NMIMS Global Access School for Continuing Education programs?  This includes:

* What are the minimum educational qualifications required for each program?
* Are there any specific entrance exams required? If so, which ones and what are their requirements?
* What is the minimum work experience required (if any) for each program?
* Are there any age restrictions?
* What are the English language proficiency requirements (e.g., TOEFL, IELTS scores)?
* Are there any other specific criteria, such as prior knowledge or skills needed for specific programs?
* What is the application process?
* What documents are required for application?
* What are the selection criteria used by the admissions committee?
* What is the process for evaluating applications?
* Are there any specific criteria for scholarships or financial aid?
* What are the deadlines for application submission?
* What is the process for appealing a decision?



## Keywords Extractor

In [ ]:
import requests

def generate_answer_with_gemini_KEYWORD(system_prompt, query, api_key):
    # print('Context: ', context)
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json",
        # "Authorization": f"Bearer {api_key}"
    }
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"SYSTEM_PROMPT: {system_prompt}\n\n USER'S_QUESTION: {query}\n\n KEYWORDS:"
                    }
                ]
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json().get('candidates')[0].get('content').get('parts')[0].get('text',"Sorry, I didn’t understand your question. Do you want to connect with a live agent?")
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
api_key = "api_key"
# failure_statement = "Sorry, I didn't understand your question. Do you want to connect with a live agent?"
user_query = input()

print("Answer:", answer)

Answer: KEYWORDS: admitted, student, kit, contents



## Generate an Answer

In [ ]:
import requests
def generate_answer_with_gemini_KEYWORD(system_prompt, query, api_key):
    # print('Context: ', context)
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json",
        # "Authorization": f"Bearer {api_key}"
    }
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"SYSTEM_PROMPT: {system_prompt}\n\n USER'S_QUESTION: {query}\n\n"
                    }
                ]
            }
        ],
         "generation_config": {
            "temperature": 0.0
        }
    }
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json().get('candidates')[0].get('content').get('parts')[0].get('text',"Sorry, I didn’t understand your question. Do you want to connect with a live agent?")
    else:
        return f"Error: {response.status_code} - {response.text}"

def generate_answer_with_gemini(system_prompt, query, context, api_key):
    print('Context: ', context)
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json",
        # "Authorization": f"Bearer {api_key}"
    }
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": f"SYSTEM_PROMPT: {system_prompt}\n\n CONTEXT: {context}\n\n USER'S_QUERY: {query}\n\n ANSWER:"
                    }
                ]
            }
        ],
         "generation_config": {
            "temperature": 0.0
        }
    }
    response = requests.post(url, headers=headers, json=data)
    # print(response.status_code)
    # return response.json().get('candidates')[0].get('content').get('parts')[0].get('text')
    if response.status_code == 200:
        return response.json().get('candidates')[0].get('content').get('parts')[0].get('text')
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
api_key = "api_key"
# failure_statement = "Sorry, I didn't understand your question. Do you want to connect with a live agent?"
System_Prompt = f"You are an AI chat bot to answer user questions on Student Resource Book of the NMIMS Global Access School For Continuing Education. I have provided you with the relevent chunks of information to answer the user's question under the tag {"CONTEXT"}. Following the CONTEXT I have provided you with the user's question inder the tag {"USER'S_QUESTION"}. Make sure the user's question is comprehendable and make's sense, if you are unsure of what the user is asking for, reply requesting for more information by the user. Answer the user's question using the only the context provided and do not use any other information. Do not Hallucinate. If the CONTEXT provided does not contain the information relevant to answer the user's question, reply with this STATEMENT: {"Sorry, I didn't understand your question. Do you want to connect with a live agent?"}."
System_Prompt_KEYWORD = f"You are an AI Agent to help retrieve information from Student Resource Book of the NMIMS Global Access School For Continuing Education to answer user questions on that database. Extract the relevant Keywords from the user query on which according to whic the information will be extracted from the database. Provide me with those Keywords."
user_query = input()
# relevant_chunk = find_relevant_chunk(user_query, index, chunks, model)
keywords = generate_answer_with_gemini_KEYWORD(System_Prompt_KEYWORD, user_query, api_key)
print(keywords)
relevant_chunk = find_relevant_chunk(keywords, index, chunks, model)

answer = generate_answer_with_gemini(System_Prompt, user_query, relevant_chunk, api_key)
print("Answer:", answer)

Criteria

Context:  ['•List of Documents for Submission: Candidate needs to submit the self-aVested photo copies of the following mandatory documents men;oned below: As per Eligibility Criteria HSC (10+2) in any discipline from a recognized Board with minimum 50%.', 'Eligibility Criteria for registering for Placement Ac;vi;es: Course Comple9on Criteria: 90% of courses 9ll Semester IV should be successfully completed at the 9me of registra9on for placement ac9vi9es (in Semester V). 46100% of courses 9ll Semester V should be successfully completed at the 9me of commencement of Job Search (in Semester VI). Marks Criteria: Minimum 60% required at the 9me of registra9on for placements (Aggregate of Term I to IV) in Semester V.', 'Please Note: There is no individual cut- oﬀ or individual passing criteria. Aggregate marks: 50/100 or more in each subject. 30A student enrolled in the respec9ve program (as applicable) will be declared ‘Pass’ afer he/she fulﬁls the above passing criteria for all 